In [4]:
def read_fasta_file(filename):
    """
    Reads the given FASTA file f and returns a dictionary of sequences.

    Lines starting with ';' in the FASTA file are ignored.
    """
    sequences_lines = {}
    current_sequence_lines = None
    with open(filename) as fp:
        for line in fp:
            line = line.strip()
            if line.startswith(';') or not line:
                continue
            if line.startswith('>'):
                sequence_name = line.lstrip('>')
                current_sequence_lines = []
                sequences_lines[sequence_name] = current_sequence_lines
            else:
                if current_sequence_lines is not None:
                    current_sequence_lines.append(line)
    sequences = {}
    for name, lines in sequences_lines.items():
        sequences[name] = ''.join(lines)
    return sequences

In [14]:
def plus4(streng,ann,obs):
    res={}
    d=ann
    e=d.find(streng)
    index=0
    while (e!=-1):
        f=obs[index+e+1:index+e+4]
        if f in res:
            res[f]=res[f]+1
        else:
            res[f]=1
        d=d[e+4:]
        index+=e+4
        e=d.find(streng)
    return res

def plus3(streng,ann,obs):
    res={}
    d=ann 
    g=d.find(streng)
    index=0
    while (g!=-1):
        h=obs[index+g:index+g+3]
        if h in res:
            res[h]=res[h]+1
        else:
            res[h]=1
        d=d[g+4:] 
        index+=g+4
        g=d.find(streng) 
    return res

a=[]
b=[]
startc={}
slutc={}
startrc={}
slutrc={}

for i in range(1,2):
    a.append(read_fasta_file("genome"+str(i)+".fa")["genome"+str(i)])
    b.append(read_fasta_file("annotation"+str(i)+".fa")["annotation"+str(i)])
    startc=plus4("NCCC",b[i-1],a[i-1])
    slutc=plus3("CCCN",b[i-1],a[i-1])
    startrc=plus3("RRRN",b[i-1],a[i-1])
    slutrc=plus4("NRRR",b[i-1],a[i-1])
        
        
print("startcodon")
for i in startc:
    print (i,startc[i])
print("slutcodon")
for i in slutc:
    print(i,slutc[i])
print("startreversecodon")
for i in startrc:
    print(i,startrc[i])
print("slutreversecodon")
for i in slutrc:
    print(i,slutrc[i])



startcodon
GTG 35
TTG 43
ATG 731
slutcodon
TAG 167
TGA 119
TAA 523
startreversecodon
CAA 30
CAC 29
CAT 580
slutreversecodon
TCA 79
TTA 425
CTA 135


In [12]:
#training by counting
SYMBOL_DICT={'A':0,'C':1,'G':2,'T':3}
NUMBER_OF_SYMBOLS=4
import numpy as np
numberOfStates=43
A=np.zeros([numberOfStates,numberOfStates])
startVector=np.zeros([numberOfStates])
startVector[0]=1.0
emissionTabel=np.zeros([numberOfStates,NUMBER_OF_SYMBOLS])

def updateEmissionTabel(observations,states):
    for i in range(len(states)):
        emissionTabel[states[i],SYMBOL_DICT[observations[i]]]+=1
        
def counting(a,b):
    j=0
    jo=-1
    state=0
    while j <len(a):
        if j==jo:
            break;
        print("her",state,i,j,b[j:j+4],a[j:j+4])
        jo=j
        if state==0:
            if b[j]=='C':
                if a[j:j+3]=="TTG": #state=1
                    A[0,1]=A[0,1]+1
                    state=3
                    j+=3
                elif a[j:j+3]=="GTG": #state=4
                    A[0,4]+=1
                    state=6
                    j+=3
                elif a[j:j+3]=="ATG": #state=7
                    A[0,7]+=1
                    state=9
                    j+=3
                else:
                    A[0,0]+=1
                    emissionTabel[0,SYMBOL_DICT[a[j]]]+=1
                    j+=1
            elif b[j]=='R':
                if a[j:j+3]=="CTA": #state=22
                    A[0,22]+=1
                    state=24
                    j+=3
                elif a[j:j+3]=="TTA": #state=25
                    A[0,25]+=1
                    state=27
                    j+=3
                elif a[j:j+3]=="TCA": #state=28
                    A[0,28]+=1
                    state=30
                    j+=3
                else:
                    A[0,0]+=1
                    emissionTabel[0,SYMBOL_DICT[a[j]]]+=1
                    j+=1
                    
            else:
                A[0,0]+=1
                emissionTabel[0,SYMBOL_DICT[a[j]]]+=1
                j+=1
        elif (state>=10) & (state<=12):
            if b[j:j+4]=="CCCN":
                if state!=12:
                    print("UPS 4",i,j,b[j:j+4],a[j:j+4])
                else:
                    if a[j:j+3]=="TAG": #state=13
                        A[12,13]+=1
                        state=15
                        j+=3
                    elif a[j:j+3]=="TGA": #state=16
                        A[12,16]+=1
                        state=18
                        j+=3
                    elif a[j:j+3]=="TAA": #state=19
                        A[12,19]+=1
                        state=21
                        j+=3
                    else:
                        print("UPS 3",i,j,b[j:j+4],a[j:j+4])
                        state=0
            elif b[j:j+4]=="CCCC":
                updateEmissionTabel(a[j:j+3],[10,11,12])
                A[12,10]+=1
                state=12
                j+=3
            else:
                print("UPS 1",i,j,b[j:j+4],a[j:j+4])
        elif (state>=31) & (state<=33):
            if b[j:j+4]=="RRRN":
                if state!=33:
                    print("UPS 5",i,j,b[j:j+4],a[j:j+4])
                else:
                    if a[j:j+3]=="CAC": #state=34
                        A[33,34]+=1
                        state=36
                        j+=3
                    elif a[j:j+3]=="CAT": #state=37
                        A[33,37]+=1
                        state=39
                        j+=3
                    elif a[j:j+3]=="CAA": #state=40
                        A[33,40]+=1
                        state=42
                        j+=3
                    else:
                        print("UPS 6",i,j,b[j:j+4],a[j:j+4])
            elif b[j:j+4]=="RRRR":
                updateEmissionTabel(a[j:j+3],[31,32,33])
                state=33
                j+=3
            else:
                print("UPS 2",i,j,b[j:j+4],a[j:j+4]) 
        elif (state==36) | (state==39)|(state==42)| (state==15) | (state==18)|(state==21):
            if b[j]=='N':
                A[state,0]+=1
                emissionTabel[0,SYMBOL_DICT[a[j]]]+=1
                state=0
                j+=1
            else:
                print("UPS 7",i,j,b[j],a[j]) 
        elif (state==3) | (state==6)|(state==9):
            if b[j:j+4]=="CCCC":
                updateEmissionTabel(a[j:j+3],[10,11,12])
                state=12
                j+=3
            else:
                print("UPS 9",i,j,b[j:j+4],a[j:j+4])
        elif (state==24) | (state==27)|(state==30):
            if b[j:j+4]=="RRRR":
                updateEmissionTabel(a[j:j+3],[31,32,33])
                state=33
                j+=3
            else:
                print("UPS 10",i,j,b[j:j+4],a[j:j+4])
            
        else:
            print("UPS 8",i,j,b[j],a[j]) 

                
            


In [13]:
for i in range(2,3):
    a=read_fasta_file("genome"+str(i)+".fa")["genome"+str(i)][263000:]
    b=read_fasta_file("annotation"+str(i)+".fa")["annotation"+str(i)][263000:]
#    print(a[45140:45150])
    counting(a,b)
c=np.sum(A,axis=1,keepdims=True)

print(c)
    

her 0 2 0 NNNN TCAA
her 0 2 1 NNNN CAAA
her 0 2 2 NNNN AAAT
her 0 2 3 NNNN AATT
her 0 2 4 NNNN ATTG
her 0 2 5 NNNN TTGT
her 0 2 6 NNNN TGTT
her 0 2 7 NNNN GTTT
her 0 2 8 NNNN TTTG
her 0 2 9 NNNN TTGT
her 0 2 10 NNNN TGTT
her 0 2 11 NNNN GTTA
her 0 2 12 NNNN TTAT
her 0 2 13 NNNN TATC
her 0 2 14 NNNN ATCA
her 0 2 15 NNNN TCAA
her 0 2 16 NNNN CAAG
her 0 2 17 NNNN AAGA
her 0 2 18 NNNN AGAA
her 0 2 19 NNNN GAAT
her 0 2 20 NNNN AATT
her 0 2 21 NNNN ATTT
her 0 2 22 NNNN TTTG
her 0 2 23 NNNN TTGA
her 0 2 24 NNNN TGAA
her 0 2 25 NNNN GAAA
her 0 2 26 NNNN AAAA
her 0 2 27 NNNN AAAT
her 0 2 28 NNNN AATA
her 0 2 29 NNNN ATAT
her 0 2 30 NNNN TATG
her 0 2 31 NNNN ATGG
her 0 2 32 NNNN TGGA
her 0 2 33 NNNN GGAC
her 0 2 34 NNNN GACT
her 0 2 35 NNNN ACTG
her 0 2 36 NNNN CTGG
her 0 2 37 NNNN TGGC
her 0 2 38 NNNN GGCT
her 0 2 39 NNNN GCTC
her 0 2 40 NNNN CTCT
her 0 2 41 NNNN TCTA
her 0 2 42 NNNN CTAG
her 0 2 43 NNNN TAGG
her 0 2 44 NNNN AGGC
her 0 2 45 NNNN GGCA
her 0 2 46 NNNN GCAA
her 0 2 47 NNNN CAAA
he